In [14]:
from sortedcontainers import SortedList


class MyClass:
    def __init__(self, first, second):
        self.first = first
        self.second = second

    def __lt__(self, other):
        return self.first < other.first or (self.first == other.first and self.second < other.second)

    def __eq__(self, other):
        return self.first == other.first and self.second == other.second

    def __hash__(self):
        return hash((self.first, self.second))


In [16]:
sl = SortedList()
a, b, c = MyClass(1, 2), MyClass(1, 4), MyClass(1, 0)
sl.add(a)
sl.add(b)
sl.add(c)
print(sl.index(a), sl.index(b),sl.index(c))

1 2 0
